In [7]:
!pip install sentence-transformers --quiet

In [18]:
%%sql
DROP DATABASE IF EXISTS movie_recommender;
CREATE DATABASE IF NOT EXISTS movie_recommender;

++
||
++
++

In [28]:
%%sql
CREATE TABLE IF NOT EXISTS tags (
    `userId` bigint(20) NULL,
    `movieId` bigint(20) NULL,
    `tag` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
    `timestamp` bigint(20) NULL
);

CREATE PIPELINE tags
    AS LOAD DATA S3 'studiotutorials/movielens/tags.csv'
    CONFIG '{\"region\":\"us-east-1\", \"disable_gunzip\": false}'
    BATCH_INTERVAL 2500
    MAX_PARTITIONS_PER_BATCH 1
    DISABLE OUT_OF_ORDER OPTIMIZATION
    DISABLE OFFSETS METADATA GC
    SKIP DUPLICATE KEY ERRORS
    INTO TABLE `tags`
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' ESCAPED BY '\\'
    LINES TERMINATED BY '\r\n'
    NULL DEFINED BY ''
    IGNORE 1 LINES
    (userId, movieId, tag, timestamp);

START PIPELINE tags;

++
||
++
++

In [29]:
%%sql
CREATE TABLE IF NOT EXISTS ratings (
    userId bigint(20) DEFAULT NULL,
    movieId bigint(20) DEFAULT NULL,
    rating double DEFAULT NULL,
    timestamp bigint(20) DEFAULT NULL
);

CREATE PIPELINE ratings
    AS LOAD DATA S3 'studiotutorials/movielens/ratings.csv'
    CONFIG '{\"region\":\"us-east-1\", \"disable_gunzip\": false}'
    BATCH_INTERVAL 2500
    MAX_PARTITIONS_PER_BATCH 1
    DISABLE OUT_OF_ORDER OPTIMIZATION
    DISABLE OFFSETS METADATA GC
    SKIP DUPLICATE KEY ERRORS
    INTO TABLE `ratings`
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' ESCAPED BY '\\'
    LINES TERMINATED BY '\r\n'
    NULL DEFINED BY ''
    IGNORE 1 LINES
    (userId, movieId, rating, timestamp);

START PIPELINE ratings;

++
||
++
++

In [30]:
%%sql
CREATE TABLE movies (
    movieId bigint(20) DEFAULT NULL,
    title text CHARACTER SET utf8 COLLATE utf8_general_ci,
    genres text CHARACTER SET utf8 COLLATE utf8_general_ci,
    FULLTEXT(title)
);

CREATE PIPELINE movies
    AS LOAD DATA S3 'studiotutorials/movielens/movies.csv'
    CONFIG '{\"region\":\"us-east-1\", \"disable_gunzip\": false}'
    BATCH_INTERVAL 2500
    MAX_PARTITIONS_PER_BATCH 1
    DISABLE OUT_OF_ORDER OPTIMIZATION
    DISABLE OFFSETS METADATA GC
    SKIP DUPLICATE KEY ERRORS
    INTO TABLE `movies`
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' ESCAPED BY '\\'
    LINES TERMINATED BY '\r\n'
    NULL DEFINED BY ''
    IGNORE 1 LINES
    (movieId, title, genres);

START PIPELINE movies;

++
||
++
++

Check that all the data has been loaded

In [31]:
%%sql
SELECT COUNT(*) AS count_rows FROM ratings
UNION ALL
SELECT COUNT(*) AS count_rows FROM movies
UNION ALL
SELECT COUNT(*) AS count_rows FROM tags


count_rows
0
62423
1093360


Concatenate tags and movies tables using all tags

In [32]:
%%sql
CREATE TABLE movies_with_tags AS
    SELECT
        m.movieId,
        m.title,
        m.genres,
        GROUP_CONCAT(t.tag SEPARATOR ',') AS allTags
    FROM movies m
    LEFT JOIN tags t ON m.movieId = t.movieId
    GROUP BY m.movieId, m.title, m.genres;

++
||
++
++

3. Vectorize data

In [34]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [98]:
%%sql result <<
SELECT * FROM movies_with_tags

Convert the result from the above SQL into a DataFrame and clean up quotes.

In [99]:
import pandas as pd

df = pd.DataFrame(result)

# Curate the special characters
df['title'] = df['title'].str.replace('"', '')
df['allTags'] = df['allTags'].str.replace('"', '').str.replace("'", '')

data = df.to_dict(orient='records')

In [100]:
data[0]

{'movieId': 58103,
 'title': 'Vantage Point (2008)',
 'genres': 'Action|Drama|Thriller',
 'allTags': 'assassination,assassination,assassination,Ensemble Cast,US President,spain,complot,Eduardo Noriega,usa president,assassination,brain bang,William Hurt,secret service,car chases,car journey,terrorist,limousine,hypertextual (mildly),camcorder,bombs,multiple storylines,secret service,multiple storylines,secret service,assassination,multiple storylines,explosions,spain,bombs,murder,fast paced,betrayal,president,secret_service,multiple storylines,assassin,To See,betrayal,fast paced,assassin,seeAlso:Death of a President (2006),assassin,terrorism,secret service,salamanca,double agents,terror,security guard,secret service,double agents,president,bodyguard,secret service,secret service,double agents,double agents,Bodrio,ending,Pete Travis,television,fast paced,multiple storylines,Forest Whitaker,murder,terrorism,multiple storylines,terrorism,US President,Dennis Quaid,multiple storylines,spain,s

In [57]:
all_title_type_column = [f'{row["title"]}-{row["allTags"]}' if row["title"] is not None else row["title"]for row in data]


Create the Embeddings

In [58]:
# Remove [:3000] if you want to vectorize all rows (~60 minutes)
all_embeddings = model.encode(all_title_type_column[:3000])
all_embeddings.shape

(3000, 768)

In [101]:
# Remember the list will be only 3,000 elements
for row, embedding in zip(data, all_embeddings):
    row['embedding'] = embedding

In [102]:
data[0]

{'movieId': 58103,
 'title': 'Vantage Point (2008)',
 'genres': 'Action|Drama|Thriller',
 'allTags': 'assassination,assassination,assassination,Ensemble Cast,US President,spain,complot,Eduardo Noriega,usa president,assassination,brain bang,William Hurt,secret service,car chases,car journey,terrorist,limousine,hypertextual (mildly),camcorder,bombs,multiple storylines,secret service,multiple storylines,secret service,assassination,multiple storylines,explosions,spain,bombs,murder,fast paced,betrayal,president,secret_service,multiple storylines,assassin,To See,betrayal,fast paced,assassin,seeAlso:Death of a President (2006),assassin,terrorism,secret service,salamanca,double agents,terror,security guard,secret service,double agents,president,bodyguard,secret service,secret service,double agents,double agents,Bodrio,ending,Pete Travis,television,fast paced,multiple storylines,Forest Whitaker,murder,terrorism,multiple storylines,terrorism,US President,Dennis Quaid,multiple storylines,spain,s

4. Create table for movie information and vectors

In [66]:
%%sql
DROP TABLE IF EXISTS movie_with_tags_with_vectors;

CREATE TABLE movie_with_tags_with_vectors (
    movieId BIGINT(20) DEFAULT NULL,
    title text CHARACTER SET utf8 COLLATE utf8_general_ci,
    genres text CHARACTER SET utf8 COLLATE utf8_general_ci,
    allTags longtext CHARACTER SET utf8mb4,
    vector BLOB
)

++
||
++
++

Create a database connection using SQLAlchemy. We are going to use an SQLAlchemy connection here because one column of data is numpy arrays. The SingleStoreDB SQLAlchemy driver will automatically convert those to the correct binary format when uploading, so it's a bit more convenient than doing the conversions and formatting manually for the %sql magic command.

In [103]:
from singlestoredb import create_engine

conn = create_engine().connect()

In [104]:
import sqlalchemy as sa

sql_query = sa.text('''
    INSERT INTO movie_with_tags_with_vectors (
        movieId,
        title,
        genres,
        allTags,
        vector
    )
    VALUES (
        :movieId,
        :title,
        :genres,
        :allTags,
        :embedding
    )
    ''')

conn.execute(sql_query, data[:10])

5. Build Semantic Search

In [105]:
%%sql
WITH queryouter AS (
                SELECT DISTINCT(title), movieId, MATCH(title) AGAINST ('Pocahontas*') as relevance
                FROM movies
                WHERE MATCH(title) AGAINST ('Pocahontas*')
                ORDER BY relevance DESC
                LIMIT 10)
    SELECT title, movieId FROM queryouter;

title,movieId
Pocahontas (1995),48


Create favorite movie tables for user

In [106]:
%%sql
CREATE ROWSTORE TABLE IF NOT EXISTS user_choice (
    userid text CHARACTER SET utf8 COLLATE utf8_general_ci,
    title text CHARACTER SET utf8 COLLATE utf8_general_ci,
    ts datetime DEFAULT NULL,
    KEY userid (userid)
)


++
||
++
++

In [72]:
%%sql
INSERT INTO user_choice (userid, title, ts)
    VALUES ('user1', 'Zone 39 (1997)', '2022-01-01 00:00:00'),
           ('user1', 'Star Trek II: The Wrath of Khan (1982)', '2022-01-01 00:00:00'),
           ('user1', 'Giver, The (2014)', '2022-01-01 00:00:00');

++
||
++
++

Build semantic search for a movie recommendation

In [108]:
%%sql
WITH
    table_match AS (
        SELECT
            m.title,
            m.movieId,
            m.vector
        FROM
            user_choice t
            INNER JOIN movie_with_tags_with_vectors m ON m.title = t.title
        WHERE
            userid = 'user1'
    ),
    movie_pairs AS (
        SELECT
            m1.movieId AS movieId1,
            m1.title AS title1,
            m2.movieId AS movieId2,
            m2.title AS title2,
            DOT_PRODUCT(m1.vector, m2.vector) AS similarity
        FROM
            table_match m1
            CROSS JOIN movie_with_tags_with_vectors m2
        WHERE
            m1.movieId != m2.movieId
            AND NOT EXISTS (
                SELECT
                    1
                FROM
                    user_choice uc
                WHERE
                    uc.userid = 'user1'
                    AND uc.title = m2.title
            )
    ),
    movie_match AS (
        SELECT
            movieId1,
            title1,
            movieId2,
            title2,
            similarity
        FROM
            movie_pairs
        ORDER BY
            similarity DESC
    ),
    distinct_count AS (
        SELECT DISTINCT
            movieId2,
            title2 AS Title,
            ROUND(AVG(similarity), 4) AS Rating_Match
        FROM
            movie_match
        GROUP BY
            movieId2,
            title2
        ORDER BY
            Rating_Match DESC
    ),
    average_ratings AS (
        SELECT
            movieId,
            AVG(rating) AS Avg_Rating
        FROM
            ratings
        GROUP BY
            movieId
    )
SELECT
    dc.Title,
    dc.Rating_Match as 'Match Score',
    ROUND(ar.Avg_Rating, 1) AS 'Average User Rating'
FROM
    distinct_count dc
    JOIN average_ratings ar ON dc.movieId2 = ar.movieId
ORDER BY
    dc.Rating_Match DESC
LIMIT
    5;


Title,Match Score,Average User Rating


What are you looking for??

In [109]:
search_embedding = model.encode("I want see a French comedy movie")

In [110]:
sql_query = sa.text('''
    SELECT title, genres, DOT_PRODUCT(vector, :vector) AS score FROM movie_with_tags_with_vectors tv
    ORDER BY Score DESC
    LIMIT 10
''')

results = conn.execute(sql_query, dict(vector=search_embedding))

for i, res in enumerate(results):
   print(f"{i + 1}: {res.title} {res.genres} Score: {res.score}")

1: The Night Visitor (1971) Crime|Horror|Thriller Score: 0.3107210099697113
2: Cat and Mouse (1974) Action|Drama Score: 0.29090893268585205
3: Sands of the Kalahari (1965) Action|Adventure Score: 0.2876102030277252
4: A Mother Should Be Loved (1934) (no genres listed) Score: 0.2749166190624237
5: Leprechaun 4: In Space (1997) Comedy|Fantasy|Horror|Sci-Fi Score: 0.2511323094367981
6: Cleopatra (1970) Animation|Comedy|Fantasy|Sci-Fi Score: 0.2375088334083557
7: Daughters of the Sun (2000) Drama Score: 0.23156818747520447
8: Vantage Point (2008) Action|Drama|Thriller Score: 0.22034500539302826
9: Time Freak (2018) Comedy|Drama|Sci-Fi Score: 0.1960551142692566
10: Down the Shore (2011) Drama|Romance|Thriller Score: 0.152541846036911
